<a href="https://colab.research.google.com/github/shinjangwoon/jangwoonshin/blob/practice/Kaggle_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Titanic

##Kaggle titanic data 불러오기

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
!ls -1ha kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c titanic

##시작

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


In [ ]:
# Load data
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

print(train_data.info())
print(test_data.info())

In [ ]:
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)

train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)

print(train_data['Embarked'].value_counts())
train_data['Embarked'].fillna('S', inplace = True)
test_data['Embarked'].fillna('S', inplace = True)


In [ ]:
def bar_chart(feature):
    survived = train_data[train_data['Survived'] == 1][feature].value_counts()
    dead = train_data[train_data['Survived'] == 0][feature].value_counts()
    df = pd.DataFrame([survived, dead])
    df.index = ['Survived', 'Dead']
    df.plot(kind='bar', stacked=True, figsize=(10,5))

In [ ]:
bar_chart('Pclass')

In [ ]:
bar_chart('Sex')

In [ ]:
bar_chart('SibSp')

In [ ]:
bar_chart('Parch')

In [ ]:
bar_chart('Embarked')

In [ ]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
test_features = test_data[features]

In [ ]:
dvec = DictVectorizer(sparse=False)
train_features = dvec.fit_transform(train_features.to_dict(orient='records'))
print(dvec.feature_names_)

In [ ]:
## Create Mode
# DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy')
# Training
clf.fit(train_features, train_labels)
# Prediction
test_features = dvec.fit_transform(test_features.to_dict(orient='records'))
prediction = clf.predict(test_features)
# Calculation accuracy
acc_decision_tree = round(clf.score(train_features, train_labels), 6)
print(u'score is %.4lf' % acc_decision_tree)

# Verification accuracy
print(u'cross_val_score is %.4lf' % np.mean(cross_val_score(clf, train_features, train_labels, cv=10)))

In [ ]:
submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": prediction
})

submission.to_csv('submission.csv', index=False)
submission = pd.read_csv('submission.csv')
submission.head()